In [5]:
using SparseArrays
using Plots
function solver(Mat::SparseMatrixCSC{Float64, Int64},b::Vector{Float64})
    if(size(Mat)[1]==size(Mat)[2])
        if(size(Mat)[1]!=size(b)[1])
            error("Wrong dimensions")
        end
        u = Mat \ b
        return u::Vector{Float64}
    else
        error("It's not a square matrix")
    end
end

function sourceFct(x)
    mu = 0.3
    sigma = .1
    return exp( -(x-mu)^2/sigma^2)
end 

N = 10; 
h = 1/N;
xstart = 0. 
xend = 1.
x = Vector(xstart:h:xend)
f = sourceFct.(x)
h2 = h*h;
Np1 = N+1; 
# initialize to zero matrix 
A = zeros(Np1,Np1)

# loop over rows and columns to set values of matrix A 
for i in axes(A,1), j in axes(A,2)
    if (i==j) A[i,j] = 2/h2 end 
    if ((i==j-1) || (i==j+1)) A[i,j] = -1/h2 end 
end
# modify the first row of matrix A 
A[1,1] = 1; A[1,2] = 0; 
# modify the last row of matrix A 
A[end,end] = 1; A[end,end-1] = 0;  

B = A 
B[5,5] = 3.4 
A = convert(SparseMatrixCSC{Float64, Int64},A)
u=solver(A,f)
plot(x,u)
using BenchmarkTools
@code_warntype solver(A,f)

MethodInstance for solver(::SparseMatrixCSC{Float64, Int64}, ::Vector{Float64})
  from solver(Mat::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}) @ Main In[5]:3
Arguments
  #self#::Core.Const(solver)
  Mat::SparseMatrixCSC{Float64, Int64}
  b::Vector{Float64}
Locals
  u::Vector{Float64}
Body::Vector{Float64}
1 ─       Core.NewvarNode(:(u))
│   %2  = Main.size(Mat)::Tuple{Int64, Int64}
│   %3  = Base.getindex(%2, 1)::Int64
│   %4  = Main.size(Mat)::Tuple{Int64, Int64}
│   %5  = Base.getindex(%4, 2)::Int64
│   %6  = (%3 == %5)::Bool
└──       goto #5 if not %6
2 ─ %8  = Main.size(Mat)::Tuple{Int64, Int64}
│   %9  = Base.getindex(%8, 1)::Int64
│   %10 = Main.size(b)::Tuple{Int64}
│   %11 = Base.getindex(%10, 1)::Int64
│   %12 = (%9 != %11)::Bool
└──       goto #4 if not %12
3 ─       Main.error("Wrong dimensions")
4 ┄       (u = Mat \ b)
│   %16 = u::Vector{Float64}
│   %17 = Core.apply_type(Main.Vector, Main.Float64)::Core.Const(Vector{Float64})
│   %18 = Core.typeassert(%16, %17):

In [6]:
using LinearSolve

function solv(Mat,b)
    Mat = convert(SparseMatrixCSC{Float64, Int64}, Mat)
    b = convert(Vector{Float64},b)
    prob = LinearProblem(Mat, b)
    if(size(Mat)[1]==size(Mat)[2])
        if(size(Mat)[1]!=size(b)[1])
            error("Wrong dimensions")
        end
        u = solve(prob, KrylovJL_GMRES())
        return u
    else
        error("It's not a square matrix")
    end
end
@btime solv(A,f)

  3.008 μs (49 allocations: 5.50 KiB)


u: 11-element Vector{Float64}:
  0.0001234098040866799
  0.002431581943428087
  0.00455659769388216
  0.003002819032621802
 -0.008550959628638562
 -0.006972346071709938
 -0.005576888903668656
 -0.0041826658336682416
 -0.0027884438890195744
 -0.0013942219445097865
  5.242885663363542e-22